In [1]:
# Dependencies
!pip3 install --user awscli
!pip3 install --user boto3

In [2]:
from pyspark import SparkContext
from geopyspark import geopyspark_conf
from geopyspark.geotrellis import Extent
from geopyspark.geotrellis.layer import Pyramid
from geopyspark.geotrellis.color import ColorMap
from geopyspark.geotrellis.constants import *
import json
import shapely
from shapely.geometry import shape, MultiPolygon

conf = geopyspark_conf("local[*]", "MVP")
pysc = SparkContext.getOrCreate(conf)

In [3]:
import boto3
client = boto3.client("s3")
# https://boto3.readthedocs.io/en/latest/reference/customizations/s3.html#boto3.s3.transfer.S3Transfer
transfer = boto3.s3.transfer.S3Transfer(client)

bucket = "raster-vision"
key_prefix = "datasets/spacenet/AOI_3_Paris/AOI_3_Paris_Train/geojson/buildings/"
!mkdir -p /tmp/spacenet-data
filename_prefix = "/tmp/spacenet-data/"

# Returns only up to 1000 of the objects in a bucket at a time
names = []
temp_names = [key['Key'] for key in client.list_objects(Bucket=bucket, Prefix=key_prefix)['Contents']]
while len(temp_names) == 1000:
    last_name = temp_names.pop()
    names += temp_names
    temp_names = [key['Key'] for key in client.list_objects(Marker=last_name, Bucket=bucket, Prefix=key_prefix)['Contents']]

names += temp_names

# only get file names from full paths returned by list_objects
filenames = [filename_prefix + name[name.find("buildings_AOI_3_Paris_"):] for name in names]

# Download takes 1 minute.
for name, filename in zip(names, filenames):
    transfer.download_file(
        bucket=bucket,
        key=name,
        filename=filename
    )

In [4]:
polygon_list = []
for filename in filenames:
    with open(filename, encoding='utf-8') as data_file:
        buildings = json.load(data_file)
        features = buildings['features']
        for geom in features:
            geometry = geom['geometry']
            the_shape = shape(geometry)
            polygon_list.append(the_shape)

paris_buildings = MultiPolygon(polygon_list)

print(len(paris_buildings))

16455


In [5]:
from functools import partial
import fiona
import json
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

paris_buildings = transform(project, paris_buildings)

In [6]:
# Try to be friedly with python libraries like shapely
from geopyspark import geotrellis
Extent(*paris_buildings.bounds)

Extent(xmin=243702.2666430303, ymin=6271096.898148117, xmax=256007.82661225073, ymax=6284607.938010221)

In [7]:
# All execution time here is sending WKB over py4j socket
from geopyspark.geotrellis.rasterize import rasterize
from geopyspark.geotrellis import RasterizerOptions

ro = RasterizerOptions(includePartial=True, sampleType='PixelIsArea')

building_raster = rasterize(pysc, geoms=paris_buildings.geoms, 
                        crs="EPSG:3857", zoom=19, 
                        fill_value=1, cell_type="float32",
                        options = ro,
                        numPartitions = 20)

print(building_raster.layer_metadata.bounds)

Bounds(minKey=SpatialKey(col=265332, row=179924), maxKey=SpatialKey(col=265493, row=180101))


## Show Rasterized Roads on a Map

In [8]:
M.set_center(2.263631399938739, 48.98803590003459, 19)

In [9]:
# Pyramid up from base layer
building_pp = building_raster.pyramid(start_zoom=19, end_zoom=1).cache()

In [11]:
from geopyspark.geotrellis.tms import *
from geonotebook.wrappers.raster import TMSRasterData

# color map buildings 1 to red
buildings_cm = ColorMap.from_colors(pysc, [1], [0x0000ff])

# start JVM tile server and serve tiles to map
server = rdd_tms_server(pysc, building_pp, buildings_cm)
M.add_layer(TMSRasterData(server), name="TMS")

s3server = s3_catalog_tms_server(pysc, "azavea-research-emr", "spacenet-viz/ingest/yoni", "catalog", buildings_cm)

Added TMS server at host 0:0:0:0:0:0:0:0
Added TMS server at port 54953
